<a href="https://www.kaggle.com/code/rubanzasilva/fastai-neural-network-gradient-boosting?scriptVersionId=190267753" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Binary Classification of Insurance Cross 

The objective of this competition is to predict which customers respond positively to an automobile insurance offer.

A company offering health insurance to a segment of customers wants to find out whether their health insurance customers are willing to also get/sign up for their motor vehicle insurance plan.

Submissions are evaluated on area under the ROC curve metric using the predicted probabilities and ground truth targets. This means we shall always have to return the predicted probabilities for each class. Probability predictions can be more informative than outright class predictions.

In my submissions below I use the probabilities of the second class by using [:, 1] on the resulting probability predictions. This selects the probability of the positive class.



The above metric is suited to binary classification because it measures the model's ability to distinguish between classes.

## Library Imports

In [ ]:
%pip install catboost
%pip install optuna
%pip install optuna_distributed
%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
#%pip install -q -U autogluon.tabular
#%pip install autogluon
#ip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#%pip freeze > requirements.txt

In [ ]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv





import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
#from autogluon.tabular import TabularDataset, TabularPredictor

import gc

from xgboost import plot_importance

#from IPython.display import FileLink

In [ ]:
!ls /kaggle/input/

In [ ]:
!ls /kaggle/input/health-insurance-cross-sell-prediction-data

In [ ]:
path = Path('/kaggle/input/playground-series-s4e7/')
path

In [ ]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
original_train_df = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction-data/train.csv',index_col='id')
original_test_df = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction-data/test.csv',index_col='id')
                                
#original_df = pd.read_csv('/kaggle/input/academic-success-dataset/data.csv',delimiter=';')

In [ ]:
train_df.shape,test_df.shape,original_train_df.shape,original_test_df.shape

## Subset

In [ ]:
train_subset = train_df.sample(n=1000000,replace=False)
test_subset = test_df.sample(n=500000,replace=False)

In [ ]:
train_subset.shape,test_subset.shape

In [ ]:
train_subset.shape,test_subset.shape

In [ ]:
cont_names,cat_names = cont_cat_split(train_subset, dep_var='Response')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_subset))
to = TabularPandas(train_subset, procs=[Categorify, FillMissing,Normalize],
                  cat_names = cat_names,
                  cont_names = cont_names,
                  y_names='Response',
                  y_block=CategoryBlock(),
                  splits=splits)
dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_subset)
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

# Full Dataset

Below i use the fastai cont_cat_split which returns the values of a particular dataframe as either categorical or continous values depending on the cardinality of the column values which is how many levels a particular column has.

This function takes an argument of max_card whose default is 20. If the number of unique values is above the max_card, then that variables is considered to be continuos and vice versa.

In [ ]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='Response')
len(cat_names),len(cont_names)

In [ ]:
cont_names

In [ ]:
cat_names

Below, i use fastai's RandomSplitter to divide my dataset into a train and validation set, in this example it creates a validation set with 20% of the training dataset.


In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

Below i build a TabularPandas Object which is a subclass of the Tabular class that has methods we can use to handle tabular data preparation and preprocessing.

This object takes in arguments where i define my training dataset and preprocessing steps i.e. Categorify, FillMissing,Normalize.

We also define our categorical and continuous variables resulting from the cont_cat_split.
Furthermore, i define my Target variable, the y which in this case is Response, y_block represents the type of problem.

Lastly we have the splits argument which was defined above and represents the dataset split.

In [ ]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Response',
                   y_block=CategoryBlock(),
                   splits=splits)

In [ ]:
to.xs.iloc[:2]

In [ ]:
dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_df)

In [ ]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [ ]:
dls.show_batch()

# Exploratory Data Analysis

Below are the feature descriptions as described in this [notebook](https://www.kaggle.com/code/satyaprakashshukl/insurance-boost-analysis)

Gender: Categorical variable indicating the gender of the customer.

Age: Numeric variable indicating the age of the customer.

Driving_License: Binary variable indicating if the customer has a driving license (1 if yes, 0 if no).

Region_Code: Numeric variable indicating the region code of the customer.

Previously_Insured: Binary variable indicating if the customer was previously insured (1 if yes, 0 if no).

Vehicle_Age: Categorical variable indicating the age of the vehicle.

Vehicle_Damage: Categorical variable indicating if the vehicle was damaged in the past.

Annual_Premium: Numeric variable indicating the annual premium amount.

Policy_Sales_Channel: Numeric variable indicating the sales channel of the policy.

Vintage: Numeric variable indicating the number of days the customer has been associated with the company.

Response: Binary target variable indicating if the customer responded positively to the automobile insurance offer (1 if yes, 0 if no).

### Summary Statistics

In [ ]:
train_df.shape,test_df.shape

In [ ]:
train_df.head()

In [ ]:
cont_names

In [ ]:
cat_names

In [ ]:
len(cat_names),len(cont_names)

In [ ]:
train_df.info()

In [ ]:
train_df.columns

Null Value Check

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df.hist(figsize=(20,15),edgecolor='black');

In [ ]:
train_df.describe().T

In [ ]:
train_df.describe(include=[object])

### Target

In [ ]:
train_df['Response'].hist(figsize=(6,4));

In [ ]:
train_df['Response'].value_counts()

In [ ]:
train_df['Response'].unique()

In [ ]:
#barplot
sns.countplot(x='Response', data=train_df, palette='winter')
plt.figure(figsize=(6, 4)) ;
plt.show();

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

# Count the occurrences of each response
response_counts = train_df['Response'].value_counts()

# Create a pie chart
plt.figure(figsize=(3, 3))
colors = sns.color_palette('winter', n_colors=len(response_counts))
plt.pie(response_counts.values, labels=response_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)
plt.title('Distribution of Response')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

The target variable is imbalanced heavily skewing towards 0 which represents a no.

### Categorical Variables

In [ ]:
cat_names

#### Vehicle Damage

In [ ]:
train_df['Vehicle_Damage'].hist(figsize=(6,4));

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

# Count the occurrences of each response
vd_counts = train_df['Vehicle_Damage'].value_counts()

# Create a pie chart
plt.figure(figsize=(3, 3))
colors = sns.color_palette('winter', n_colors=len(vd_counts))
plt.pie(vd_counts.values, labels=vd_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)
plt.title('Distribution of Vehicle_Damage')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

In [ ]:
contingency_table = pd.crosstab(train_df['Vehicle_Damage'], train_df['Response'])
print(contingency_table)

In [ ]:
sns.countplot(x='Vehicle_Damage', hue='Response', data=train_df)
plt.show()

#### Gender

In [ ]:
train_df['Gender'].hist(figsize=(6,4));

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

# Count the occurrences of each response
gender_counts = train_df['Gender'].value_counts()

# Create a pie chart
plt.figure(figsize=(3, 3))
colors = sns.color_palette('winter', n_colors=len(gender_counts))
plt.pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)
plt.title('Distribution of Gender')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

In [ ]:
sns.countplot(x='Gender', hue='Response', data=train_df)
plt.show()

### Continous Variables

In [ ]:
cont_names

#### Age

In [ ]:
train_df['Age'].hist(figsize=(6,4));

In [ ]:
sns.boxplot(x='Age', data=train_df)
plt.show()
plt.close()


In [ ]:
sns.histplot(x = 'Age', data = train_df)

In [ ]:
# Change the display options
pd.set_option('display.float_format', '{:.4f}'.format)

# Now run your describe() function
print(train_df['Age'].describe())

In [ ]:
pd.reset_option('display.float_format')

In [ ]:
train_df.Age.mode()

Outlier check

In [ ]:
train_df.Age.var()


Age and response

### Correlation

In [ ]:
# Extract data from fastai DataLoaders
df = pd.DataFrame(dls.train.dataset.items)

# Select numeric columns
numeric_df = df.select_dtypes(include=[np.number])

# Replace inf values with NaN if any
#numeric_df = numeric_df.replace([np.inf, -np.inf], np.nan)

# Calculate correlation matrix
correlation_matrix = numeric_df.corr()

# Create the plot
plt.figure(figsize=(20, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".1f", linewidths=0.1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#correlation_matrix = .corr()
plt.figure(figsize=(20, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f", linewidths=0.1)
#plt.gcf().set_facecolor('skyblue')  
plt.title('Correlation Matrix')
plt.show()

#### Theories

From the correlation matrix, i infer that vehicle damage and Age have a heavy influence on the Response.
We notice that people with vehicle damage are way more likely to buy car insuarance.

In [ ]:
dls

In [ ]:
to

In [ ]:
dls.info()

# Neural Network

In [ ]:
learn = tabular_learner(dls, metrics=RocAucBinary())

In [ ]:
learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
%%time
learn.fit_one_cycle(8,0.005)


In [ ]:
dl = test_dl
nn_preds = learn.get_preds(dl=dl)
nn_preds_x = learn.get_preds()[0]
a_preds, _ = learn.get_preds(dl=dl)
nn_preds_y = a_preds.squeeze(1)
nn_preds_proba = (a_preds[:, 1])

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = nn_preds_proba
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

# Neural Network Ensemble

In [ ]:
def ensemble():
    learn = tabular_learner(dls, metrics=RocAucMulti())
    with learn.no_bar(),learn.no_logging(): learn.fit(5, 0.005)
    return learn.get_preds(test_dl=test_dl)[0]

In [ ]:
learns = [ensemble() for _ in range(5)]

In [ ]:
ens_preds = torch.stack(learns).mean(0)

# Random Forest

In [ ]:
%%time
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))
rf_preds_probs = tensor(rf_model.predict_proba(test_dl.xs))[:, 1]

rf_preds_x = tensor(rf_model.predict(X_test))
rf_preds_proba = tensor(rf_model.predict_proba(X_test))[:, 1]

#mse = mean_absolute_error(y_test, rf_preds_x)
#rmse = np.sqrt(mse)

#accuracy_score(y_test,rf_preds_x)
rf_score = roc_auc_score(y_test,rf_preds_proba)
rf_score

In [ ]:
%%time
#After dropping Driving_License column
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))
rf_preds_probs = tensor(rf_model.predict_proba(test_dl.xs))[:, 1]

rf_preds_x = tensor(rf_model.predict(X_test))
rf_preds_proba = tensor(rf_model.predict_proba(X_test))[:, 1]

#mse = mean_absolute_error(y_test, rf_preds_x)
#rmse = np.sqrt(mse)

#accuracy_score(y_test,rf_preds_x)
rf_score = roc_auc_score(y_test,rf_preds_proba)
rf_score

In [ ]:
%%time
#After dropping Driving_License,Gender column
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))
rf_preds_probs = tensor(rf_model.predict_proba(test_dl.xs))[:, 1]

rf_preds_x = tensor(rf_model.predict(X_test))
rf_preds_proba = tensor(rf_model.predict_proba(X_test))[:, 1]

#mse = mean_absolute_error(y_test, rf_preds_x)
#rmse = np.sqrt(mse)

#accuracy_score(y_test,rf_preds_x)
rf_score = roc_auc_score(y_test,rf_preds_proba)
rf_score

In [ ]:
%%time
#After dropping Driving_License,Gender,Vehicle_Age column
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))
rf_preds_probs = tensor(rf_model.predict_proba(test_dl.xs))[:, 1]

rf_preds_x = tensor(rf_model.predict(X_test))
rf_preds_proba = tensor(rf_model.predict_proba(X_test))[:, 1]

#mse = mean_absolute_error(y_test, rf_preds_x)
#rmse = np.sqrt(mse)

#accuracy_score(y_test,rf_preds_x)
rf_score = roc_auc_score(y_test,rf_preds_proba)
rf_score

In [ ]:
#RandomForestClassifier??

## Feature Importance

In [ ]:
def rf_feat_importance(m, train_subset):
    return pd.DataFrame({'cols':train_subset.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feat_importance(rf_model, X_train)
#fi[:10]

fi

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

#plot_fi(fi[:30]);
plot_fi(fi);
     

#### Investigate effect of dropping columns

In [ ]:
# Dropping a single column
train_subset = train_subset.drop('Driving_License', axis=1)

# Dropping multiple columns
#train_subset_new = train_subset.drop(['Driving_License', 'Gender', 'Vehicle_Age'], axis=1)

In [ ]:
# Dropping a single column
train_subset = train_subset.drop('Gender', axis=1)


In [ ]:
# Dropping a single column
train_subset = train_subset.drop('Vehicle_Age', axis=1)


In [ ]:
train_subset.shape

In [ ]:
train_subset.columns

# Gradient Boosting

# XGBoost

In [ ]:
print(xgb.get_config())

In [ ]:
xgb_params = {
              
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'booster': 'gbtree',
    'n_estimators': 1500,       
    'learning_rate': 0.55,                   
    'min_child_weight': 17,      
    'reg_lambda': 0.2,           
    'reg_alpha': 7,              
    'max_bin': 52000,            
    'colsample_bytree': 0.65,    
    'max_delta_step': 2,         
    #'random_state': 0,
    #'device' : 'gpu',
    'device' : 'cuda',
    'tree_method': 'hist'
}


In [ ]:
%%time
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))
xgb_preds_proba = tensor(xgb_model.predict_proba(test_dl.xs))[:, 1]

xgb_preds_x = tensor(xgb_model.predict(X_test))
xgb_preds_x_proba = tensor(xgb_model.predict_proba(X_test))[:, 1]

xgb_score = roc_auc_score(y_test,xgb_preds_x_proba)
xgb_score

In [ ]:
xgb_score

In [ ]:
import gc
gc.collect()

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = xgb_preds_proba
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

In [ ]:
!tree

#### XgBoost feature importance

In [ ]:
#plot_importance(xgb_model.fit(X_train, y_train))
plot_importance(xgb_model)

## Optuna

In [ ]:
%%time
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 2000),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 30),  
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'max_bin': trial.suggest_int('max_bin', 40000, 60000),
        'max_delta_step': trial.suggest_int('max_delta_step', 1, 10),
        'device_type': 'gpu'
        
    }

    model =  xgb.XGBClassifier(**params, random_state=0)
    
    # Cross-validation with 5 folds using KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kf, scoring='roc_auc')
    
    # We maximize accuracy, so we return the mean accuracy of the cross-validation
    return np.mean(cv_results)

study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [ ]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

### Cross validation implementation

In [ ]:
%%time

K_FOLDS = 5  # Number of folds for cross-validation
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

# This part is just to demonstrate how the indices work
for train_index, test_index in kfold.split(train_df):
    print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

fold_scores = []

for train_index, val_index in kfold.split(X_train):
    X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    
    # Train the model on the current fold
    Xgb_model_fold = xgb.XGBClassifier(**xgb_params)
    Xgb_model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict probabilities on the validation set for the current fold
    y_pred_fold = Xgb_model_fold.predict_proba(X_fold_val)[:, 1]
    y_pred_fold_tt = Xgb_model_fold.predict_proba(test_dl.xs)[:, 1]
    
    # Calculate and store the AUC-ROC score for the current fold
    score = roc_auc_score(y_fold_val, y_pred_fold)
    fold_scores.append(score)

# Calculate the average score across all folds
avg_score = np.mean(fold_scores)
print(f"Average AUC-ROC Score Across All Folds: {avg_score}")

In [ ]:
y_pred_fold_tt.shape

In [ ]:
avg_score

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = y_pred_fold_tt
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

# LightGBM

In [ ]:
lgbm_params = {'n_estimators': 1190, 'learning_rate': 0.22952000374471332, 'max_depth': 13, 'reg_alpha': 8.200152384535924,
          'reg_lambda': 4.285393733702208, 
          'num_leaves': 100, 'subsample': 0.6497981764924947, 'colsample_bytree': 0.37368304607248115,\
               'device_type':'gpu'}

In [ ]:
%%time
#ds subset
lgbm_model = lgb.LGBMClassifier(**lgbm_params)
lgbm_model = lgbm_model.fit(X_train, y_train)

#test set preds
lgbm_preds = tensor(lgbm_model.predict(test_dl.xs))
lgbm_preds_prob = tensor(lgbm_model.predict_proba(test_dl.xs))
lgbm_positive_preds = (lgbm_preds_prob[:, 1])

#validation set preds
lgbm_preds_x = tensor(lgbm_model.predict(X_test))
lgbm_preds_x_prob = tensor(lgbm_model.predict_proba(X_test))
lgbm_positive_preds_x = (lgbm_preds_x_prob[:, 1])

lgbm_score = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score

#lgb_preds_x_prob = tensor(lgb_model.predict_proba(X_test))

lgbm_score_prob = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score_prob


In [ ]:
lgbm_score_prob

In [ ]:
%%time

K_FOLDS = 5  # Number of folds for cross-validation
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

# This part is just to demonstrate how the indices work
for train_index, test_index in kfold.split(train_df):
    print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

fold_scores = []

for train_index, val_index in kfold.split(X_train):
    X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    
    # Train the model on the current fold
    lgb_model_fold = lgb.LGBMClassifier(**params)
    lgb_model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict probabilities on the validation set for the current fold
    y_pred_fold = lgb_model_fold.predict_proba(X_fold_val)[:, 1]
    y_pred_fold_tt = lgb_model_fold.predict_proba(test_dl.xs)[:, 1]
    
    # Calculate and store the AUC-ROC score for the current fold
    score = roc_auc_score(y_fold_val, y_pred_fold)
    fold_scores.append(score)

# Calculate the average score across all folds
avg_score = np.mean(fold_scores)
print(f"Average AUC-ROC Score Across All Folds: {avg_score}")

## Hyperparamter optimization with Optuna

In [ ]:
# finetuned optuna code
%%time

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'device_type': 'gpu'
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    fold_scores = []
    
    #for train_index, test_index in kfold.split(train_df):
    #print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

    for train_index, val_index in kfold.split(X_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        lgb_model_fold = lgb.LGBMClassifier(**params)  # Use params, not lgbm_params
        lgb_model_fold.fit(X_fold_train, y_fold_train)
        
        # Predict probabilities on the validation set for the current fold
        y_pred_fold = lgb_model_fold.predict_proba(X_fold_val)[:, 1]
        
        # Calculate and store the AUC-ROC score for the current fold
        score = roc_auc_score(y_fold_val, y_pred_fold)
        fold_scores.append(score)
    
    # Calculate the average score across all folds
    return np.mean(fold_scores)

# Create and run the study
study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [ ]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [ ]:
lgbm_optuna_run2_params  =  {
    'num_leaves': 320, 
    'learning_rate': 0.13981961408994045, 
    'n_estimators': 843,
    'subsample_for_bin': 172567, 
    'min_child_samples': 223,
    'reg_alpha': 0.0028770084050677926,
    'reg_lambda': 2.3761404778025532e-05,
    'colsample_bytree': 0.9350638004692479,
    'subsample': 0.9727470703757719,
    'max_depth': 6,
    'device_type':'gpu'
}

### With Optuna tuned parameters

In [ ]:
%%time
#ds subset
lgbm_model = lgb.LGBMClassifier(**lgbm_optuna_run2_params)
lgbm_model = lgbm_model.fit(X_train, y_train)

#test set preds
lgbm_preds = tensor(lgbm_model.predict(test_dl.xs))
lgbm_preds_prob = tensor(lgbm_model.predict_proba(test_dl.xs))
lgbm_positive_preds = (lgbm_preds_prob[:, 1])

#validation set preds
lgbm_preds_x = tensor(lgbm_model.predict(X_test))
lgbm_preds_x_prob = tensor(lgbm_model.predict_proba(X_test))
lgbm_positive_preds_x = (lgbm_preds_x_prob[:, 1])

lgbm_score = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score

#lgb_preds_x_prob = tensor(lgb_model.predict_proba(X_test))

lgbm_score_prob = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score_prob


In [ ]:
lgbm_score_prob


In [ ]:
%%time

K_FOLDS = 5  # Number of folds for cross-validation
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

# This part is just to demonstrate how the indices work
for train_index, test_index in kfold.split(train_df):
    print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

fold_scores = []

for train_index, val_index in kfold.split(X_train):
    X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    
    # Train the model on the current fold
    lgb_model_fold = lgb.LGBMClassifier(**lgbm_optuna_run2_params)
    lgb_model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict probabilities on the validation set for the current fold
    y_pred_fold = lgb_model_fold.predict_proba(X_fold_val)[:, 1]
    y_pred_fold_tt = lgb_model_fold.predict_proba(test_dl.xs)[:, 1]
    
    # Calculate and store the AUC-ROC score for the current fold
    score = roc_auc_score(y_fold_val, y_pred_fold)
    fold_scores.append(score)

# Calculate the average score across all folds
avg_score = np.mean(fold_scores)
print(f"Average AUC-ROC Score Across All Folds: {avg_score}")

In [ ]:
avg_score

# CatBoost

In [ ]:
cat_optuna_params = {
    
    'colsample_bylevel': 0.6383474716497279,
    'learning_rate': 0.075,
    'max_bin': 490,
    'depth': 9,
    'l2_leaf_reg': 0.5,
    'subsample': 0.8429457747642737,
    'eval_metric':'AUC',
    'class_names':[0, 1],
    'iterations':5000,
    'random_strength':0,
    'max_leaves':512,
    'fold_permutation_block':64,
    'task_type': 'GPU',
    'devices': '0:1'
    
}

In [ ]:
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'class_names': [0, 1],
    'learning_rate': 0.075,
    'iterations': 3000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'max_leaves': 512,
    'fold_permutation_block': 64,
    'task_type': 'GPU',
    'devices': '0:1',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False
}

In [ ]:
%%time
#using full ds
cat_model = CatBoostClassifier(**cat_params)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))
cat_preds_probs = tensor(cat_model.predict_proba(test_dl.xs))[:, 1]
#cat_preds_final = cat_preds.squeeze(1)

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))
cat_preds_x_proba = tensor(cat_model.predict_proba(X_test))[:, 1]

#cat_preds_x_final = cat_preds_x.squeeze(1)

#accuracy_score(y_test,cat_preds_x)

cat_score = roc_auc_score(y_test,cat_preds_x_proba)
cat_score

In [ ]:
cat_score

In [ ]:
%%time
#using full ds
cat_model = CatBoostClassifier(**cat_params)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))
cat_preds_probs = tensor(cat_model.predict_proba(test_dl.xs))[:, 1]
#cat_preds_final = cat_preds.squeeze(1)

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))
cat_preds_x_proba = tensor(cat_model.predict_proba(X_test))[:, 1]

#cat_preds_x_final = cat_preds_x.squeeze(1)

#accuracy_score(y_test,cat_preds_x)

cat_score = roc_auc_score(y_test,cat_preds_x_proba)
cat_score

Optuna optimization

In [ ]:
%%time
#will you graduate notebook
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 6000),
        'depth': trial.suggest_int('depth', 4, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 1e-2, 10, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.00, log=True),
        'od_type': 'Iter',
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'task_type': 'GPU',  # Set task type to GPU
        'devices': '0:1',    # Use both GPUs
        'eval_metric': 'AUC',
        'loss_function': 'Logloss',
        'class_names': [0, 1],
        #'max_leaves': trial.suggest_int('max_leaves', 300, 1000), 
        
    }

    cat_model = CatBoostClassifier(**param, verbose=0)
    cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)

    cat_preds_proba = cat_model.predict_proba(X_test)
    cat_preds_proba = (cat_preds_proba[:, 1])
    cat_proba_score = roc_auc_score(y_test,cat_preds_proba)
    return cat_proba_score

# UNCOMMENT THE FOLLOWING LINES TO RUN OPTUNA, I COMMENTED THEM OUT TO SAVE SOME TIME WHILE PUBLISHING
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

In [ ]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

[I 2024-07-28 13:10:00,811] Trial 35 finished with value: 0.8806571694660618 and parameters: {'iterations': 3922, 'depth': 7, 'learning_rate': 0.06785132188911386, 'l2_leaf_reg': 0.010246847658225403, 'border_count': 255, 'random_strength': 8.699168928480285, 'bagging_temperature': 0.07986828861909359, 'od_wait': 36}. Best is trial 35 with value: 0.8806571694660618.

[I 2024-07-28 14:53:15,848] Trial 59 finished with value: 0.880659558073007 and parameters: {'iterations': 4770, 'depth': 9, 'learning_rate': 0.0751956964415004, 'l2_leaf_reg': 0.18411119632584752, 'border_count': 248, 'random_strength': 0.1621690161205084, 'bagging_temperature': 0.3374322444670465, 'od_wait': 38}. Best is trial 59 with value: 0.880659558073007.

[I 2024-07-28 15:05:48,337] Trial 61 finished with value: 0.880751074531679 and parameters: {'iterations': 4801, 'depth': 9, 'learning_rate': 0.07092162741545756, 'l2_leaf_reg': 0.14524960916052582, 'border_count': 249, 'random_strength': 0.16969220470731253, 'bagging_temperature': 0.05031377942990587, 'od_wait': 43}. Best is trial 61 with value: 0.880751074531679.

In [ ]:
silva_optuna_cat_params = {
    'iterations': 906, 'depth': 10,
    'learning_rate': 0.09919645690545881,
    'l2_leaf_reg': 0.4254495849561265,
    'border_count': 248,
    'random_strength': 2.655971961645386,
    'bagging_temperature': 0.0913538359007028,
    'od_wait': 46,
    'task_type': 'GPU',  # Set task type to GPU
    'devices': '0:1',    # Use both GPUs
    'eval_metric': 'AUC',
}

In [ ]:
silva_optuna_cat_params

In [ ]:
%%time
#using full ds
cat_model = CatBoostClassifier(**silva_optuna_cat_params)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))
cat_preds_probs = tensor(cat_model.predict_proba(test_dl.xs))[:, 1]
#cat_preds_final = cat_preds.squeeze(1)

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))
cat_preds_x_proba = tensor(cat_model.predict_proba(X_test))[:, 1]

#cat_preds_x_final = cat_preds_x.squeeze(1)

#accuracy_score(y_test,cat_preds_x)

cat_score = roc_auc_score(y_test,cat_preds_x_proba)
cat_score

In [ ]:
cat_score

# Model Predictions

In [ ]:
model_preds = {
    "random forests":roc_auc_score(y_test,rf_preds_x),
    "cat boost":cat_score,
    "lgbm":lgb_score,
    "xgboost":roc_auc_score(y_test,xgb_preds_x),   
}

#model_preds_a = model_preds.sort()
print(model_preds)

In [ ]:
cat_preds_x

# Model Ensemble

# Submission

In [ ]:
!ls

In [ ]:
lgbm_preds_final = (lgbm_preds_prob[:, 1])

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = lgbm_preds_final 
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

In [ ]:
!ls

# OpenFE

In [70]:
ofe = OpenFE()

In [71]:
#set n_jobs to be the actual cpu core count
CPU_COUNT = os.cpu_count()
n_jobs = CPU_COUNT
n_jobs


4

In [72]:
train_subset.shape

(1000000, 11)

In [73]:
train_subset.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
2904941,Male,25,1,9.0,0,< 1 Year,Yes,33990.0,152.0,166,0
7879547,Female,21,1,28.0,1,< 1 Year,No,49514.0,152.0,84,0
1554558,Female,31,1,15.0,0,< 1 Year,No,36913.0,152.0,149,0
5792501,Male,25,1,10.0,1,< 1 Year,No,36274.0,152.0,256,0
7511622,Male,66,1,28.0,1,1-2 Year,Yes,44214.0,26.0,223,0


In [74]:
%%time
features = ofe.fit(data=train_subset.drop('Response', axis=1), label=train_subset['Response'], feature_boosting=True, n_jobs=n_jobs)

[LightGBM] [Info] Number of positive: 98301, number of negative: 701699
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.187546 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 735
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122876 -> initscore=-1.965470
[LightGBM] [Info] Start training from score -1.965470
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[63]	valid_0's binary_logloss: 0.256062
[LightGBM] [Info] Number of positive: 98301, number of negative: 701699
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 734
[LightGBM] [Info] Number of data poin

  0%|          | 0/16 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 12090, number of negative: 87910
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002740 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

  6%|▋         | 1/16 [00:53<13:26, 53.77s/it]

[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 1
[LightGBM] [Info] Number of positive: 12090, number of negative: 87910
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002701 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of posi

...

[LightGBM] [Info] Number of positive: 98301, number of negative: 701699
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014997 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 124
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 1
[LightGBM] [Info] Number of positive: 98301, number of negative: 701699
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 204
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 1
[LightGBM] [Info] Number of positive: 98301, number of negative: 701699
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 47
[LightGBM] [In

100%|██████████| 16/16 [12:38<00:00, 47.41s/it]


The number of remaining candidate features is 241
Start stage II selection.


100%|██████████| 16/16 [07:28<00:00, 28.02s/it]


Finish data processing.
[LightGBM] [Info] Number of positive: 98301, number of negative: 701699
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.175070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30052
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 248
CPU times: user 19min 26s, sys: 22.3 s, total: 19min 49s
Wall time: 31min 24s


In [75]:
train_x, test_x = transform(train_subset.drop('Response', axis=1), test_subset, features, n_jobs=n_jobs)

In [ ]:
#train_x, test_x = transform(train_df.drop('Response', axis=1), test_df, features, n_jobs=4)

In [76]:
train_x.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,autoFE_f_0,autoFE_f_1,autoFE_f_2,autoFE_f_3,autoFE_f_4,autoFE_f_5,autoFE_f_6,autoFE_f_7,autoFE_f_8,autoFE_f_9,autoFE_f_10,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,autoFE_f_20,autoFE_f_21,autoFE_f_22,autoFE_f_23,autoFE_f_24,autoFE_f_25,autoFE_f_26,autoFE_f_27,autoFE_f_28,autoFE_f_29,autoFE_f_30,autoFE_f_31,autoFE_f_32,autoFE_f_33,autoFE_f_34,autoFE_f_35,autoFE_f_36,autoFE_f_37,autoFE_f_38,autoFE_f_39,autoFE_f_40,autoFE_f_41,autoFE_f_42,autoFE_f_43,autoFE_f_44,autoFE_f_45,autoFE_f_46,autoFE_f_47,autoFE_f_48,autoFE_f_49,autoFE_f_50,autoFE_f_51,autoFE_f_52,autoFE_f_53,autoFE_f_54,autoFE_f_55,autoFE_f_56,autoFE_f_57,autoFE_f_58,autoFE_f_59,autoFE_f_60,autoFE_f_61,autoFE_f_62,autoFE_f_63,autoFE_f_64,autoFE_f_65,autoFE_f_66,autoFE_f_67,autoFE_f_68,autoFE_f_69,autoFE_f_70,autoFE_f_71,autoFE_f_72,autoFE_f_73,autoFE_f_74,autoFE_f_75,autoFE_f_76,autoFE_f_77,autoFE_f_78,autoFE_f_79,autoFE_f_80,autoFE_f_81,autoFE_f_82,autoFE_f_83,autoFE_f_84,autoFE_f_85,autoFE_f_86,autoFE_f_87,autoFE_f_88,autoFE_f_89,autoFE_f_90,autoFE_f_91,autoFE_f_92,autoFE_f_93,autoFE_f_94,autoFE_f_95,autoFE_f_96,autoFE_f_97,autoFE_f_98,autoFE_f_99,autoFE_f_100,autoFE_f_101,autoFE_f_102,autoFE_f_103,autoFE_f_104,autoFE_f_105,autoFE_f_106,autoFE_f_107,autoFE_f_108,autoFE_f_109,autoFE_f_110,autoFE_f_111,autoFE_f_112,autoFE_f_113,autoFE_f_114,autoFE_f_115,autoFE_f_116,autoFE_f_117,autoFE_f_118,autoFE_f_119,autoFE_f_120,autoFE_f_121,autoFE_f_122,autoFE_f_123,autoFE_f_124,autoFE_f_125,autoFE_f_126,autoFE_f_127,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137,autoFE_f_138,autoFE_f_139,autoFE_f_140,autoFE_f_141,autoFE_f_142,autoFE_f_143,autoFE_f_144,autoFE_f_145,autoFE_f_146,autoFE_f_147,autoFE_f_148,autoFE_f_149,autoFE_f_150,autoFE_f_151,autoFE_f_152,autoFE_f_153,autoFE_f_154,autoFE_f_155,autoFE_f_156,autoFE_f_157,autoFE_f_158,autoFE_f_159,autoFE_f_160,autoFE_f_161,autoFE_f_162,autoFE_f_163,autoFE_f_164,autoFE_f_165,autoFE_f_166,autoFE_f_167,autoFE_f_168,autoFE_f_169,autoFE_f_170,autoFE_f_171,autoFE_f_172,autoFE_f_173,autoFE_f_174,autoFE_f_175,autoFE_f_176,autoFE_f_177,autoFE_f_178,autoFE_f_179,autoFE_f_180,autoFE_f_181,autoFE_f_182,autoFE_f_183,autoFE_f_184,autoFE_f_185,autoFE_f_186,autoFE_f_187,autoFE_f_188,autoFE_f_189,autoFE_f_190,autoFE_f_191,autoFE_f_192,autoFE_f_193,autoFE_f_194,autoFE_f_195,autoFE_f_196,autoFE_f_197,autoFE_f_198,autoFE_f_199,autoFE_f_200,autoFE_f_201,autoFE_f_202,autoFE_f_203,autoFE_f_204,autoFE_f_205,autoFE_f_206,autoFE_f_207,autoFE_f_208,autoFE_f_209,autoFE_f_210,autoFE_f_211,autoFE_f_212,autoFE_f_213,autoFE_f_214,autoFE_f_215,autoFE_f_216,autoFE_f_217,autoFE_f_218,autoFE_f_219,autoFE_f_220,autoFE_f_221,autoFE_f_222,autoFE_f_223,autoFE_f_224,autoFE_f_225,autoFE_f_226,autoFE_f_227,autoFE_f_228,autoFE_f_229,autoFE_f_230,autoFE_f_231,autoFE_f_232,autoFE_f_233,autoFE_f_234,autoFE_f_235,autoFE_f_236,autoFE_f_237,autoFE_f_238,autoFE_f_239,autoFE_f_240
openfe_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2904941,Male,25,1,9.0,0,< 1 Year,Yes,33990.0,152.0,166,5642340.0,0.480421,186.0,5109.0,542950.0,0.178632,166.0,305910.0,0.0,166.0,166.0,0.653033,NaN,0.380000,14.485116,0.529460,0.510935,0.280615,33990.0,191.0,6910.0,0.573447,34156.0,0.537776,33990.0,34015.0,1359.600000,0.509686,0.500451,0.487574,0.0,34.986475,0.589180,223.618421,0.499175,17590.0,6078.0,28.0,78.856796,175.0,895.0,14079.233812,0.643177,6846.0,16.888889,0.625551,0.162050,0.0,26902.168033,30648.0,0.0,0.554006,125.991393,33965.0,0.0,0.225569,0.499973,166.0,5166480.0,NaN,0.0,0.505359,79.646454,127.0,0.0,6.640000,165

In [77]:
train_x.shape

(1000000, 251)

In [78]:
train_x.reset_index(drop=True, inplace=True)

In [79]:
train_x

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,autoFE_f_0,autoFE_f_1,autoFE_f_2,autoFE_f_3,autoFE_f_4,autoFE_f_5,autoFE_f_6,autoFE_f_7,autoFE_f_8,autoFE_f_9,autoFE_f_10,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,autoFE_f_20,autoFE_f_21,autoFE_f_22,autoFE_f_23,autoFE_f_24,autoFE_f_25,autoFE_f_26,autoFE_f_27,autoFE_f_28,autoFE_f_29,autoFE_f_30,autoFE_f_31,autoFE_f_32,autoFE_f_33,autoFE_f_34,autoFE_f_35,autoFE_f_36,autoFE_f_37,autoFE_f_38,autoFE_f_39,autoFE_f_40,autoFE_f_41,autoFE_f_42,autoFE_f_43,autoFE_f_44,autoFE_f_45,autoFE_f_46,autoFE_f_47,autoFE_f_48,autoFE_f_49,autoFE_f_50,autoFE_f_51,autoFE_f_52,autoFE_f_53,autoFE_f_54,autoFE_f_55,autoFE_f_56,autoFE_f_57,autoFE_f_58,autoFE_f_59,autoFE_f_60,autoFE_f_61,autoFE_f_62,autoFE_f_63,autoFE_f_64,autoFE_f_65,autoFE_f_66,autoFE_f_67,autoFE_f_68,autoFE_f_69,autoFE_f_70,autoFE_f_71,autoFE_f_72,autoFE_f_73,autoFE_f_74,autoFE_f_75,autoFE_f_76,autoFE_f_77,autoFE_f_78,autoFE_f_79,autoFE_f_80,autoFE_f_81,autoFE_f_82,autoFE_f_83,autoFE_f_84,autoFE_f_85,autoFE_f_86,autoFE_f_87,autoFE_f_88,autoFE_f_89,autoFE_f_90,autoFE_f_91,autoFE_f_92,autoFE_f_93,autoFE_f_94,autoFE_f_95,autoFE_f_96,autoFE_f_97,autoFE_f_98,autoFE_f_99,autoFE_f_100,autoFE_f_101,autoFE_f_102,autoFE_f_103,autoFE_f_104,autoFE_f_105,autoFE_f_106,autoFE_f_107,autoFE_f_108,autoFE_f_109,autoFE_f_110,autoFE_f_111,autoFE_f_112,autoFE_f_113,autoFE_f_114,autoFE_f_115,autoFE_f_116,autoFE_f_117,autoFE_f_118,autoFE_f_119,autoFE_f_120,autoFE_f_121,autoFE_f_122,autoFE_f_123,autoFE_f_124,autoFE_f_125,autoFE_f_126,autoFE_f_127,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137,autoFE_f_138,autoFE_f_139,autoFE_f_140,autoFE_f_141,autoFE_f_142,autoFE_f_143,autoFE_f_144,autoFE_f_145,autoFE_f_146,autoFE_f_147,autoFE_f_148,autoFE_f_149,autoFE_f_150,autoFE_f_151,autoFE_f_152,autoFE_f_153,autoFE_f_154,autoFE_f_155,autoFE_f_156,autoFE_f_157,autoFE_f_158,autoFE_f_159,autoFE_f_160,autoFE_f_161,autoFE_f_162,autoFE_f_163,autoFE_f_164,autoFE_f_165,autoFE_f_166,autoFE_f_167,autoFE_f_168,autoFE_f_169,autoFE_f_170,autoFE_f_171,autoFE_f_172,autoFE_f_173,autoFE_f_174,autoFE_f_175,autoFE_f_176,autoFE_f_177,autoFE_f_178,autoFE_f_179,autoFE_f_180,autoFE_f_181,autoFE_f_182,autoFE_f_183,autoFE_f_184,autoFE_f_185,autoFE_f_186,autoFE_f_187,autoFE_f_188,autoFE_f_189,autoFE_f_190,autoFE_f_191,autoFE_f_192,autoFE_f_193,autoFE_f_194,autoFE_f_195,autoFE_f_196,autoFE_f_197,autoFE_f_198,autoFE_f_199,autoFE_f_200,autoFE_f_201,autoFE_f_202,autoFE_f_203,autoFE_f_204,autoFE_f_205,autoFE_f_206,autoFE_f_207,autoFE_f_208,autoFE_f_209,autoFE_f_210,autoFE_f_211,autoFE_f_212,autoFE_f_213,autoFE_f_214,autoFE_f_215,autoFE_f_216,autoFE_f_217,autoFE_f_218,autoFE_f_219,autoFE_f_220,autoFE_f_221,autoFE_f_222,autoFE_f_223,autoFE_f_224,autoFE_f_225,autoFE_f_226,autoFE_f_227,autoFE_f_228,autoFE_f_229,autoFE_f_230,autoFE_f_231,autoFE_f_232,autoFE_f_233,autoFE_f_234,autoFE_f_235,autoFE_f_236,autoFE_f_237,autoFE_f_238,autoFE_f_239,autoFE_f_240
0,Male,25,1,9.0,0,< 1 Year,Yes,33990.0,152.0,166,5642340.0,0.480421,186.0,5109.0,542950.0,0.178632,166.0,305910.0,0.0,166.0,166.0,0.653033,NaN,0.380000,14.485116,0.529460,0.510935,0.280615,33990.0,191.0,6910.0,0.573447,34156.0,0.537776,33990.0,34015.0,1359.600000,0.509686,0.500451,0.487574,0.0,34.986475,0.589180,223.618421,0.499175,17590.0,6078.0,28.0,78.856796,175.0,895.0,14079.233812,0.643177,6846.0,16.888889,0.625551,0.162050,0.0,26902.168033,30648.0,0.0,0.554006,125.991393,33965.0,0.0,0.225569,0.499973,166.0,5166480.0,NaN,0.0,0.505359,79.646454,127.0,0.0,6.640000,165.0,849750.0,0.438852,167.0,33990.0,0.466655,177.0,34142.0,81240.0,35526.0,318.0,290594.0,1494.0,25.0,0.496267,33999.0,152.0,3800.0,33990.0,143.0,166.0,157.0,82460.0,0.998033,33838.0,80.0,225.0,0.147187,204.759036,0.0,751966.0,25.0,6063.0,33981.0,-14.0,12986.968804,3776.

In [81]:
train = pd.concat([train_x, train_subset['Response']], axis=1)

train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,autoFE_f_0,autoFE_f_1,autoFE_f_2,autoFE_f_3,autoFE_f_4,autoFE_f_5,autoFE_f_6,autoFE_f_7,autoFE_f_8,autoFE_f_9,autoFE_f_10,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,autoFE_f_20,autoFE_f_21,autoFE_f_22,autoFE_f_23,autoFE_f_24,autoFE_f_25,autoFE_f_26,autoFE_f_27,autoFE_f_28,autoFE_f_29,autoFE_f_30,autoFE_f_31,autoFE_f_32,autoFE_f_33,autoFE_f_34,autoFE_f_35,autoFE_f_36,autoFE_f_37,autoFE_f_38,autoFE_f_39,autoFE_f_40,autoFE_f_41,autoFE_f_42,autoFE_f_43,autoFE_f_44,autoFE_f_45,autoFE_f_46,autoFE_f_47,autoFE_f_48,autoFE_f_49,autoFE_f_50,autoFE_f_51,autoFE_f_52,autoFE_f_53,autoFE_f_54,autoFE_f_55,autoFE_f_56,autoFE_f_57,autoFE_f_58,autoFE_f_59,autoFE_f_60,autoFE_f_61,autoFE_f_62,autoFE_f_63,autoFE_f_64,autoFE_f_65,autoFE_f_66,autoFE_f_67,autoFE_f_68,autoFE_f_69,autoFE_f_70,autoFE_f_71,autoFE_f_72,autoFE_f_73,autoFE_f_74,autoFE_f_75,autoFE_f_76,autoFE_f_77,autoFE_f_78,autoFE_f_79,autoFE_f_80,autoFE_f_81,autoFE_f_82,autoFE_f_83,autoFE_f_84,autoFE_f_85,autoFE_f_86,autoFE_f_87,autoFE_f_88,autoFE_f_89,autoFE_f_90,autoFE_f_91,autoFE_f_92,autoFE_f_93,autoFE_f_94,autoFE_f_95,autoFE_f_96,autoFE_f_97,autoFE_f_98,autoFE_f_99,autoFE_f_100,autoFE_f_101,autoFE_f_102,autoFE_f_103,autoFE_f_104,autoFE_f_105,autoFE_f_106,autoFE_f_107,autoFE_f_108,autoFE_f_109,autoFE_f_110,autoFE_f_111,autoFE_f_112,autoFE_f_113,autoFE_f_114,autoFE_f_115,autoFE_f_116,autoFE_f_117,autoFE_f_118,autoFE_f_119,autoFE_f_120,autoFE_f_121,autoFE_f_122,autoFE_f_123,autoFE_f_124,autoFE_f_125,autoFE_f_126,autoFE_f_127,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137,autoFE_f_138,autoFE_f_139,autoFE_f_140,autoFE_f_141,autoFE_f_142,autoFE_f_143,autoFE_f_144,autoFE_f_145,autoFE_f_146,autoFE_f_147,autoFE_f_148,autoFE_f_149,autoFE_f_150,autoFE_f_151,autoFE_f_152,autoFE_f_153,autoFE_f_154,autoFE_f_155,autoFE_f_156,autoFE_f_157,autoFE_f_158,autoFE_f_159,autoFE_f_160,autoFE_f_161,autoFE_f_162,autoFE_f_163,autoFE_f_164,autoFE_f_165,autoFE_f_166,autoFE_f_167,autoFE_f_168,autoFE_f_169,autoFE_f_170,autoFE_f_171,autoFE_f_172,autoFE_f_173,autoFE_f_174,autoFE_f_175,autoFE_f_176,autoFE_f_177,autoFE_f_178,autoFE_f_179,autoFE_f_180,autoFE_f_181,autoFE_f_182,autoFE_f_183,autoFE_f_184,autoFE_f_185,autoFE_f_186,autoFE_f_187,autoFE_f_188,autoFE_f_189,autoFE_f_190,autoFE_f_191,autoFE_f_192,autoFE_f_193,autoFE_f_194,autoFE_f_195,autoFE_f_196,autoFE_f_197,autoFE_f_198,autoFE_f_199,autoFE_f_200,autoFE_f_201,autoFE_f_202,autoFE_f_203,autoFE_f_204,autoFE_f_205,autoFE_f_206,autoFE_f_207,autoFE_f_208,autoFE_f_209,autoFE_f_210,autoFE_f_211,autoFE_f_212,autoFE_f_213,autoFE_f_214,autoFE_f_215,autoFE_f_216,autoFE_f_217,autoFE_f_218,autoFE_f_219,autoFE_f_220,autoFE_f_221,autoFE_f_222,autoFE_f_223,autoFE_f_224,autoFE_f_225,autoFE_f_226,autoFE_f_227,autoFE_f_228,autoFE_f_229,autoFE_f_230,autoFE_f_231,autoFE_f_232,autoFE_f_233,autoFE_f_234,autoFE_f_235,autoFE_f_236,autoFE_f_237,autoFE_f_238,autoFE_f_239,autoFE_f_240,Response
0,Male,25.0,1.0,9.0,0.0,< 1 Year,Yes,33990.0,152.0,166.0,5642340.0,0.480421,186.0,5109.0,542950.0,0.178632,166.0,305910.0,0.0,166.0,166.0,0.653033,NaN,0.380000,14.485116,0.529460,0.510935,0.280615,33990.0,191.0,6910.0,0.573447,34156.0,0.537776,33990.0,34015.0,1359.600000,0.509686,0.500451,0.487574,0.0,34.986475,0.589180,223.618421,0.499175,17590.0,6078.0,28.0,78.856796,175.0,895.0,14079.233812,0.643177,6846.0,16.888889,0.625551,0.162050,0.0,26902.168033,30648.0,0.0,0.554006,125.991393,33965.0,0.0,0.225569,0.499973,166.0,5166480.0,NaN,0.0,0.505359,79.646454,127.0,0.0,6.640000,165.0,849750.0,0.438852,167.0,33990.0,0.466655,177.0,34142.0,81240.0,35526.0,318.0,290594.0,1494.0,25.0,0.496267,33999.0,152.0,3800.0,33990.0,143.0,166.0,157.0,82460.0,0.998033,33838.0,80.0,225.0,0.147187,204.759036,0.0,751966.0,25.0,6063.0,33981.0,-14.0,1

In [1]:
train.shape

NameError: name 'train' is not defined

In [ ]:
cont_names,cat_names = cont_cat_split(train, dep_var='Response')
splits = RandomSplitter(valid_pct=0.2)(range_of(train))
to = TabularPandas(train, procs=[Categorify, FillMissing,Normalize],
                  cat_names = cat_names,
                  cont_names = cont_names,
                  y_names='Response',
                  y_block=CategoryBlock(),
                  splits=splits)
dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_subset)
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [ ]:
#y_names='Response'

In [ ]:
features = ofe.fit(data=X_train, label=y_train, n_jobs=n_jobs) 

In [ ]:
train_x, test_x = transform(X_train, X_test, features, n_jobs=n_jobs)